# Mi cadena de markov

## Bibliotecas

In [2]:
# Warnings ---------------------------------------------------------------------------------------------------

import warnings
warnings.filterwarnings("ignore")

# Lectura y manipulación de datos ----------------------------------------------------------------------------

import pandas as pd

# Descomposición espectral -----------------------------------------------------------------------------------

import numpy as np
from numpy.linalg import eig, inv


## Lectura de la base de datos

In [3]:
data = pd.read_parquet('C:/Users/Usuario/Desktop/Estocastica/Reto/Reto-Markov/Predicting-product-demand-using-Markov-chains/data/tec_estocasticos.parquet', engine='pyarrow')

In [4]:
data

,periodo,cliente_id,material_id,tipo_cliente
0,05-2022,4894.0,22.0,Distribuidor
1,05-2022,4769.0,17.0,Distribuidor
2,05-2022,4823.0,227.0,Distribuidor
3,08-2022,4816.0,340.0,Distribuidor
4,08-2022,4888.0,270.0,Distribuidor
...,...,...,...,...
6973895,05-2022,1816.0,1861.0,Farmacia
6973896,05-2022,1725.0,1373.0,Hospital
6973897,05-2022,588.0,2249.0,Hospital
6973898,05-2022,565.0,191.0,Hospital


In [5]:
data['periodo'] = pd.to_datetime(data['periodo'])
data.sort_values(by='periodo', inplace=True)
data.dropna(inplace=True)
data.reset_index(inplace=True)
data.drop('index', axis = 1,inplace=True)
data

,periodo,cliente_id,material_id,tipo_cliente
0,2021-01-01,800.0,317.0,Hospital
1,2021-01-01,782.0,2043.0,Hospital
2,2021-01-01,303.0,1352.0,Hospital
3,2021-01-01,314.0,1577.0,Hospital
4,2021-01-01,122.0,1504.0,Hospital
...,...,...,...,...
6955591,2023-09-01,4406.0,317.0,Hospital
6955592,2023-09-01,587.0,2321.0,Hospital
6955593,2023-09-01,587.0,1359.0,Hospital
6955594,2023-09-01,2421.0,341.0,Hospital


### Tipos de Cliente

In [6]:
data['tipo_cliente'].unique()

array(['Hospital', 'Farmacia', 'Distribuidor', 'Otro'], dtype=object)

## Función para obtener la matriz de transición de un cliente comprando un determinado producto

In [7]:
def matriz_transicion(tipo_cliente, cliente_id, material_id):
    
    # Subset
    cliente_tipo = data.loc[data['tipo_cliente'] == tipo_cliente]
    pruducto = cliente_tipo.loc[cliente_tipo['material_id'] == material_id]
    id_cliente = pruducto.loc[pruducto['cliente_id'] == cliente_id]
    id_cliente.reset_index(inplace = True)
    id_cliente.drop('index', axis = 1, inplace = True)
    
    # Estados: Compró o no compró
    
    t = [0]

    for x in range(0, len(id_cliente['periodo'])-2):
        if (id_cliente['periodo'][x+1] - id_cliente['periodo'][x]).days <= 31:
            t.append(0)
        else:
            for _ in range(((id_cliente['periodo'][x+1] - id_cliente['periodo'][x]).days // 30)-1):
                t.append(1)
            t.append(0)
    
    t_1 = []

    for x in range(0, len(id_cliente['periodo'])-1):
        if (id_cliente['periodo'][x+1] - id_cliente['periodo'][x]).days <= 31:
            t_1.append(0)
        else:
            for _ in range(((id_cliente['periodo'][x+1] - id_cliente['periodo'][x]).days // 30)-1):
                t_1.append(1)
            t_1.append(0)
            

    estados = pd.DataFrame()

    estados['t'] = t
    estados['t_1'] = t_1
    
    Xt = estados['t'][0:-1].reset_index(drop=True).rename('X_t')
    Xt_1 = estados['t_1'][1::].reset_index(drop=True).rename('X_t+1')
    
    new_data=pd.concat((Xt, Xt_1), axis=1)
    
    matriz_transicion = new_data.groupby('X_t').value_counts(normalize=True).unstack(level='X_t+1')
    matriz_transicion= matriz_transicion.fillna(0)
    
    return matriz_transicion

In [9]:
 matriz_transicion('Farmacia', 2000.0, 259.0)

X_t+1,0
X_t,
0,1.0


In [24]:
matriz_transicion =  matriz_transicion('Distribuidor', 4769.0, 17.0)

## Descomposición espectral

In [8]:
## Descomposición espectral

import numpy as np
from numpy.linalg import eig, inv
Lambda, Q = eig(matriz_transicion)
print("Los eigenvalores de P son:", Lambda)
print("Los eigenvectores de P son:", Q)

LinAlgError: 0-dimensional array given. Array must be at least two-dimensional

In [ ]:
Q_1 = inv(Q)
Lambda = np.diag(Lambda)
PP = np.matmul(np.matmul(Q, Lambda), Q_1)
PP.round(decimals = 4)

## Consultar probabilidad en tiempo n

In [6]:
def pasos(n):
    Lambda_n = Lambda**n
    P_n = np.matmul(np.matmul(Q, Lambda_n), Q_1)
    
    df = pd.DataFrame(P_n.round(decimals = 4), index=['Compra', 'No compra'])
    df.rename(columns = {0:'Compra'}, inplace = True)
    df.rename(columns = {1:'No Compra'}, inplace = True)
    return df

In [38]:
pasos(5)

,Compra,No Compra
Compra,0.9033,0.0967
No compra,0.9021,0.0979


In [39]:
pasos(10)

,Compra,No Compra
Compra,0.9032,0.0968
No compra,0.9032,0.0968


In [40]:
pasos(15)

,Compra,No Compra
Compra,0.9032,0.0968
No compra,0.9032,0.0968


In [13]:
data.loc[data['tipo_cliente'] == 'Hospital']

,periodo,cliente_id,material_id,tipo_cliente
0,2021-01-01,800.0,317.0,Hospital
1,2021-01-01,782.0,2043.0,Hospital
2,2021-01-01,303.0,1352.0,Hospital
3,2021-01-01,314.0,1577.0,Hospital
4,2021-01-01,122.0,1504.0,Hospital
...,...,...,...,...
6955591,2023-09-01,4406.0,317.0,Hospital
6955592,2023-09-01,587.0,2321.0,Hospital
6955593,2023-09-01,587.0,1359.0,Hospital
6955594,2023-09-01,2421.0,341.0,Hospital


## Dropdown

In [6]:
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np

In [21]:
# ------

tipo_cliente = list(data['tipo_cliente'].unique())

cliente_id = {}

for x in tipo_cliente:
        cliente_id[x] = list(data.loc[data['tipo_cliente'] == x]['cliente_id'].unique())

boton1 = widgets.Dropdown(options=tipo_cliente, description='Tipo cliente:')
boton2 = widgets.Dropdown(options=[], description='Cliente id:')


def actualizar_opciones(*args):
    boton2.options = cliente_id[boton1.value]

boton1.observe(actualizar_opciones, 'value')


# -----

material_id = list(data['material_id'].unique()) 
      
boton3 = widgets.Dropdown(options =material_id, description='Material id:')

pasos_t = list(range(1,51))

boton4 = widgets.Dropdown(options= pasos_t, description='Pasos:')


# Función para la visualización con parámetros
def valores(tipo_cliente, cliente_id, material_id, pasos_t):
    
    try: 
        Lambda, Q = eig(matriz_transicion(tipo_cliente, cliente_id, material_id))
        Q_1 = inv(Q)
        Lambda = np.diag(Lambda)
        PP = np.matmul(np.matmul(Q, Lambda), Q_1)

        Lambda_n = Lambda**pasos_t
        P_n = np.matmul(np.matmul(Q, Lambda_n), Q_1)

        df = pd.DataFrame(P_n.round(decimals = 4), index=['Compra', 'No compra'])
        df.rename(columns = {0:'Compra'}, inplace = True)
        df.rename(columns = {1:'No Compra'}, inplace = True)
    
    except:
        return print('No existe información de ese cliente comprando ese producto')
    return print(df)


widgets.interactive(valores, 
                    tipo_cliente=boton1, 
                    cliente_id=boton2,
                    material_id=boton3,
                    pasos_t = boton4)


interactive(children=(Dropdown(description='Tipo cliente:', options=('Hospital', 'Farmacia', 'Distribuidor', '…

In [42]:
tipo_cliente = list(data['tipo_cliente'].unique())

cliente_id = {}

for x in tipo_cliente:
    cliente_id[x] = list(data.loc[data['tipo_cliente'] == x]['cliente_id'].unique())
        
        
cliente_id

{'Hospital': [800.0,
  782.0,
  303.0,
  314.0,
  122.0,
  1448.0,
  206.0,
  1676.0,
  1643.0,
  1663.0,
  1370.0,
  1001.0,
  1036.0,
  356.0,
  2091.0,
  3028.0,
  3084.0,
  2388.0,
  1215.0,
  3353.0,
  1244.0,
  1323.0,
  1150.0,
  1766.0,
  1413.0,
  2451.0,
  2484.0,
  2782.0,
  646.0,
  378.0,
  1320.0,
  2105.0,
  2410.0,
  510.0,
  3780.0,
  133.0,
  554.0,
  434.0,
  472.0,
  676.0,
  780.0,
  863.0,
  1808.0,
  1898.0,
  2052.0,
  2136.0,
  2213.0,
  2187.0,
  2497.0,
  3716.0,
  3037.0,
  561.0,
  2316.0,
  447.0,
  2359.0,
  2926.0,
  2068.0,
  1810.0,
  887.0,
  633.0,
  2993.0,
  492.0,
  2616.0,
  3688.0,
  753.0,
  1329.0,
  1558.0,
  1479.0,
  1781.0,
  2575.0,
  593.0,
  2483.0,
  1685.0,
  1873.0,
  914.0,
  450.0,
  487.0,
  1128.0,
  3077.0,
  1186.0,
  1152.0,
  135.0,
  2892.0,
  3316.0,
  2011.0,
  425.0,
  2084.0,
  2559.0,
  3228.0,
  1025.0,
  2184.0,
  2496.0,
  2281.0,
  1924.0,
  3468.0,
  1436.0,
  3150.0,
  671.0,
  3542.0,
  2002.0,
  3171.0,
  1790.0

In [18]:
material_id = {}

for x in tipo_cliente:
    for i in cliente_id[x]:
        material_id[i] = list(data.loc[data['cliente_id'] == i]['material_id'].unique())

KeyboardInterrupt: 

In [ ]:
material_id

In [17]:
cliente_id['Hospital']

[800.0,
 782.0,
 303.0,
 314.0,
 122.0,
 1448.0,
 206.0,
 1676.0,
 1643.0,
 1663.0,
 1370.0,
 1001.0,
 1036.0,
 356.0,
 2091.0,
 3028.0,
 3084.0,
 2388.0,
 1215.0,
 3353.0,
 1244.0,
 1323.0,
 1150.0,
 1766.0,
 1413.0,
 2451.0,
 2484.0,
 2782.0,
 646.0,
 378.0,
 1320.0,
 2105.0,
 2410.0,
 510.0,
 3780.0,
 133.0,
 554.0,
 434.0,
 472.0,
 676.0,
 780.0,
 863.0,
 1808.0,
 1898.0,
 2052.0,
 2136.0,
 2213.0,
 2187.0,
 2497.0,
 3716.0,
 3037.0,
 561.0,
 2316.0,
 447.0,
 2359.0,
 2926.0,
 2068.0,
 1810.0,
 887.0,
 633.0,
 2993.0,
 492.0,
 2616.0,
 3688.0,
 753.0,
 1329.0,
 1558.0,
 1479.0,
 1781.0,
 2575.0,
 593.0,
 2483.0,
 1685.0,
 1873.0,
 914.0,
 450.0,
 487.0,
 1128.0,
 3077.0,
 1186.0,
 1152.0,
 135.0,
 2892.0,
 3316.0,
 2011.0,
 425.0,
 2084.0,
 2559.0,
 3228.0,
 1025.0,
 2184.0,
 2496.0,
 2281.0,
 1924.0,
 3468.0,
 1436.0,
 3150.0,
 671.0,
 3542.0,
 2002.0,
 3171.0,
 1790.0,
 465.0,
 773.0,
 1337.0,
 631.0,
 2251.0,
 1934.0,
 739.0,
 3560.0,
 2923.0,
 3489.0,
 1906.0,
 2454.0,
 506.0,


In [40]:
# Markdown Funcional

# Opciones

tipo_cliente = list(data['tipo_cliente'].unique())

cliente_id = list(data['cliente_id'].unique()) 

material_id = list(data['material_id'].unique()) 

pasos_t = list(range(1,51))

# Botones

dropdown_tipo_cliente = widgets.Dropdown(
    options=tipo_cliente,
    description="tipo_cliente:"
)

dropdown_cliente_id = widgets.Dropdown(
    options=cliente_id,
    description="cliente_id:"
)

dropdown_material_id = widgets.Dropdown(
    options= material_id,
    description="material_id:"
)

dropdown_pasos = widgets.Dropdown(
    options=pasos_t,
    description="pasos:"
)


# Función para la visualización con parámetros
def valores(tipo_cliente, cliente_id, material_id, pasos_t):
    
    try: 
        Lambda, Q = eig(matriz_transicion(tipo_cliente, cliente_id, material_id))
        Q_1 = inv(Q)
        Lambda = np.diag(Lambda)
        PP = np.matmul(np.matmul(Q, Lambda), Q_1)

        Lambda_n = Lambda**pasos_t
        P_n = np.matmul(np.matmul(Q, Lambda_n), Q_1)

        df = pd.DataFrame(P_n.round(decimals = 4), index=['Compra', 'No compra'])
        df.rename(columns = {0:'Compra'}, inplace = True)
        df.rename(columns = {1:'No Compra'}, inplace = True)
    
    except:
        return print('No existe información de ese cliente comprando ese producto')
    return print(df)

widgets.interactive(valores, 
                    tipo_cliente=dropdown_tipo_cliente, 
                    cliente_id=dropdown_cliente_id,
                    material_id=dropdown_material_id,
                    pasos_t = dropdown_pasos)

interactive(children=(Dropdown(description='tipo_cliente:', options=('Hospital',), value='Hospital'), Dropdown…

In [12]:
import ipywidgets as widgets
from IPython.display import display

# Define las opciones iniciales para el primer y segundo botón
opciones_boton1 = ['Opción A', 'Opción B', 'Opción C']
opciones_boton2 = {
    'Opción A': ['A1', 'A2'],
    'Opción B': ['B1', 'B2', 'B3'],
    'Opción C': ['C1', 'C2', 'C3']
}

opciones_boton1 = list(data['tipo_cliente'].unique())

opciones_boton2 = {}

for x in opciones_boton1:
        opciones_boton2[x] = list(data.loc[data['tipo_cliente'] == x]['cliente_id'].unique())

# Crea los widgets para los botones desplegables
boton1 = widgets.Dropdown(options=opciones_boton1, description='Botón 1:')
boton2 = widgets.Dropdown(options=[], description='Botón 2:')

# Función para actualizar las opciones del botón 2 según la selección en el botón 1
def actualizar_opciones(*args):
    boton2.options = opciones_boton2[boton1.value]

# Vincula la función de actualización al evento de cambio en el botón 1
boton1.observe(actualizar_opciones, 'value')

# Muestra los widgets
display(boton1, boton2)


Dropdown(description='Botón 1:', options=('Hospital', 'Farmacia', 'Distribuidor', 'Otro'), value='Hospital')

Dropdown(description='Botón 2:', options=(), value=None)

In [13]:
opciones_boton2

{'Hospital': [800.0,
  782.0,
  303.0,
  314.0,
  122.0,
  1448.0,
  206.0,
  1676.0,
  1643.0,
  1663.0,
  1370.0,
  1001.0,
  1036.0,
  356.0,
  2091.0,
  3028.0,
  3084.0,
  2388.0,
  1215.0,
  3353.0,
  1244.0,
  1323.0,
  1150.0,
  1766.0,
  1413.0,
  2451.0,
  2484.0,
  2782.0,
  646.0,
  378.0,
  1320.0,
  2105.0,
  2410.0,
  510.0,
  3780.0,
  133.0,
  554.0,
  434.0,
  472.0,
  676.0,
  780.0,
  863.0,
  1808.0,
  1898.0,
  2052.0,
  2136.0,
  2213.0,
  2187.0,
  2497.0,
  3716.0,
  3037.0,
  561.0,
  2316.0,
  447.0,
  2359.0,
  2926.0,
  2068.0,
  1810.0,
  887.0,
  633.0,
  2993.0,
  492.0,
  2616.0,
  3688.0,
  753.0,
  1329.0,
  1558.0,
  1479.0,
  1781.0,
  2575.0,
  593.0,
  2483.0,
  1685.0,
  1873.0,
  914.0,
  450.0,
  487.0,
  1128.0,
  3077.0,
  1186.0,
  1152.0,
  135.0,
  2892.0,
  3316.0,
  2011.0,
  425.0,
  2084.0,
  2559.0,
  3228.0,
  1025.0,
  2184.0,
  2496.0,
  2281.0,
  1924.0,
  3468.0,
  1436.0,
  3150.0,
  671.0,
  3542.0,
  2002.0,
  3171.0,
  1790.0

In [38]:
tipo_cliente = list(data['tipo_cliente'].unique())

cliente_id = {}

for x in tipo_cliente:
    cliente_id[x] = list(data.loc[data['tipo_cliente'] == x]['cliente_id'].unique())
    
material_id = {}

for x in cliente_id.keys():
    for i in cliente_id[x]:
        material_id[i] = list(data.loc[data['cliente_id'] == i]['material_id'].unique())

In [ ]:
material_id

In [37]:
cliente_id.values()

dict_values([[800.0, 782.0, 303.0, 314.0, 122.0, 1448.0, 206.0, 1676.0, 1643.0, 1663.0, 1370.0, 1001.0, 1036.0, 356.0, 2091.0, 3028.0, 3084.0, 2388.0, 1215.0, 3353.0, 1244.0, 1323.0, 1150.0, 1766.0, 1413.0, 2451.0, 2484.0, 2782.0, 646.0, 378.0, 1320.0, 2105.0, 2410.0, 510.0, 3780.0, 133.0, 554.0, 434.0, 472.0, 676.0, 780.0, 863.0, 1808.0, 1898.0, 2052.0, 2136.0, 2213.0, 2187.0, 2497.0, 3716.0, 3037.0, 561.0, 2316.0, 447.0, 2359.0, 2926.0, 2068.0, 1810.0, 887.0, 633.0, 2993.0, 492.0, 2616.0, 3688.0, 753.0, 1329.0, 1558.0, 1479.0, 1781.0, 2575.0, 593.0, 2483.0, 1685.0, 1873.0, 914.0, 450.0, 487.0, 1128.0, 3077.0, 1186.0, 1152.0, 135.0, 2892.0, 3316.0, 2011.0, 425.0, 2084.0, 2559.0, 3228.0, 1025.0, 2184.0, 2496.0, 2281.0, 1924.0, 3468.0, 1436.0, 3150.0, 671.0, 3542.0, 2002.0, 3171.0, 1790.0, 465.0, 773.0, 1337.0, 631.0, 2251.0, 1934.0, 739.0, 3560.0, 2923.0, 3489.0, 1906.0, 2454.0, 506.0, 1301.0, 1356.0, 1430.0, 3213.0, 1428.0, 150.0, 557.0, 2663.0, 2014.0, 3499.0, 2591.0, 1478.0, 2601.0,

In [31]:
material_id

{'Hospital': [], 'Farmacia': [], 'Distribuidor': [], 'Otro': []}